# GlueX amplitude model with SymPy

Intensity function for two-pseudoscalar system:

$$
I(\Omega,\Phi) = 2\kappa\sum_{k}\left(
  (1-P_{\gamma})\left[\sum_{l,m}[l]_{m;k}^{(-)}\Re[Z_{l}^{m}(\Omega,\Phi)]\right]^2
  +(1-P_{\gamma})\left[\sum_{l,m}[l]_{m;k}^{(+)}\Im[Z_{l}^{m}(\Omega,\Phi)]\right]^2
  +(1+P_{\gamma})\left[\sum_{l,m}[l]_{m;k}^{(+)}\Re[Z_{l}^{m}(\Omega,\Phi)]\right]^2
  +(1+P_{\gamma})\left[\sum_{l,m}[l]_{m;k}^{(-)}\Im[Z_{l}^{m}(\Omega,\Phi)]\right]^2
\right)
$$

where $Z_{l}^{m}(\Omega,\Phi)=Y_{l}^{m}(\Omega)e^{-i\Phi}$ is a phase-rotated spherical harmonic, $\Omega$ is the solid angle, $\Phi$ is the angle between the production and polarization planes,  $P_{\gamma}$ is the polarization magnitude, $[l]$ are the partial wave amplitudes, $m$ is the associated m-projection, $k$ refers to a spin flip ($k=1$) or non-flip ($k=0$) at the nucleon vertex, and $\kappa$ is an overall phase space factor.

In [ ]:
from __future__ import annotations

import sympy as sp
from ampform.io import aslatex
from ampform.sympy import PoolSum
from IPython.display import Math
from sympy.functions.special.spherical_harmonics import Ynm

In [ ]:
from sympy.printing.printer import Printer


def _print_Indexed_latex(self, printer, *args):
    base = printer._print(self.base)
    indices = ", ".join(map(printer._print, self.indices))
    return f"{base}_{{{indices}}}"


sp.Indexed._latex = _print_Indexed_latex
Printer._global_settings["gothic_re_im"] = True

In [ ]:
k, l, m = sp.symbols("k l m")
phi, theta, Phi = sp.symbols("phi theta Phi")
kappa = sp.Symbol("kappa")
Py = sp.Symbol("P_gamma")

In [ ]:
from ampform.sympy import (
    UnevaluatedExpression,
    create_expression,
    implement_doit_method,
)


@implement_doit_method
class Znm(UnevaluatedExpression):
    is_commutative = True
    is_real = False

    def __new__(cls, n, m, theta, phi, Phi, **hints) -> Znm:
        return create_expression(cls, n, m, theta, phi, Phi, **hints)

    def evaluate(self) -> sp.Mul:
        n, m, theta, phi, Phi = self.args
        return Ynm(l, m, theta, phi) * sp.exp(-sp.I * Phi)

    def _latex(self, printer, *args):
        n, m, theta, phi, Phi = map(printer._print, self.args)
        return Rf"Z_{n}^{m}({theta}, {phi}, {Phi})"

In [ ]:
znm = Znm(l, m, theta, phi, Phi)
Math(aslatex({znm: znm.doit()}))

In [ ]:
assert sp.im(znm) != 0

In [ ]:
from qrules.quantum_numbers import arange


def create_range(__min, __max) -> tuple[sp.Rational, ...]:
    return tuple(sp.Rational(x) for x in arange(__min, __max))


max_l = 1
l_range = create_range(0, max_l)
m_range = create_range(-max_l, max_l)
lp = sp.IndexedBase("[l]^{(+)}", complex=True)
lm = sp.IndexedBase("[l]^{(-)}", complex=True)

intensity_expr = (
    2
    * kappa
    * PoolSum(
        (1 - Py) * (PoolSum(lm[m, k] * sp.re(znm), (l, l_range), (m, m_range))) ** 2
        + (1 - Py) * (PoolSum(lp[m, k] * sp.im(znm), (l, l_range), (m, m_range))) ** 2
        + (1 + Py) * (PoolSum(lp[m, k] * sp.re(znm), (l, l_range), (m, m_range))) ** 2
        + (1 + Py) * (PoolSum(lm[m, k] * sp.im(znm), (l, l_range), (m, m_range))) ** 2,
        (k, [-1, +1]),
    )
)
intensity_expr